In [15]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re

In [16]:
list_topic=['pol','eco','lif','soc','int']
topic_dict={'pol':"정치",'eco':"경제",'lif':"라이프",'soc':"사회",'int':"국제"}

In [17]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20200103')

In [18]:
# 각각의 리스트 생성
topic_list=[]
date_list=[]
title_list=[]
text_list=[]

In [19]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [20]:
#크롤링 def
def get_news(date,main_url,sub_url,num,topic):
    html = urlopen(main_url+date)
    bsObject = BeautifulSoup(html, "html.parser")
    bs_page=bsObject.find(class_="paging")
    for meta in bsObject.find_all('dt'):
        link=meta.a.get('href')
        title=no_space(meta.text)
        
        title_list.append(title)
        date=date
        date_list.append(date)
        url=urlopen(sub_url+link)
        bs_obj=BeautifulSoup(url, "html.parser")
        bs_text=bs_obj.body.find(class_="tx")
        text=no_space(bs_text.text)
        text_list.append(text)
        topic_list.append(topic_dict[topic])

    if(bs_page.find(class_="end").get('href')==("list.asp?sid1="+topic+"&sid2=&page=1&sdate="+date+"&st=")):
        return bsObject
  
    if(bs_page.find(class_="end").get('href')==("list.asp?sid1="+topic+"&sid2=&page="+str(num)+"&sdate="+date+"&st=")):
        return bsObject
    else:
        #재귀 호출
        url_new="http://news.kmib.co.kr/article/list.asp?sid1="+topic+"&sid2=&page="+str(num+1)+"&sdate="
        return get_news(date,url_new,sub_url,num+1,topic)


In [21]:
#크롤링 시작
try:
    for num_topic in tqdm(range(len(list_topic)),desc="Topic_Progress"):
        main_url="http://news.kmib.co.kr/article/list.asp?sid1="+list_topic[num_topic]+"&sid2=&sdate="
        sub_url='http://news.kmib.co.kr/article/'
        for num_date in tqdm(range(len(dt_index.date)),desc="Day_Progress"):
            date=dt_index.date[num_date]
            date=str(date).split('-')[0]+str(date).split('-')[1]+str(date).split('-')[2]
            bs=get_news(date,main_url,sub_url,1,list_topic[num_topic])
except:
    print("에러가 발생 했습니다.")

In [22]:
#판다스로 저장 (후에 csv파일로 저장 라인 추가)
df=pd.DataFrame({"date":date_list, "title":title_list, "topic":topic_list,"text":text_list}) 
print(df) 

         date                                          title topic  \
0    20200101                         [포토] 문 대통령, 의인들과 함께 산행    정치   
1    20200101                문희상 “공수처법 통과는 세 대통령의 꿈이 현실화한 것”    정치   
2    20200101                               북한 설맞이 화려한 축하 행사    정치   
3    20200101                  “시간이 얼마 없다” 한국당發 보수통합론 다시 불붙나    정치   
4    20200101                  문희상 의장 “공수처법 통과로 3명 대통령 원 풀어”    정치   
..        ...                                            ...   ...   
655  20200103              “트럼프 오판에 北·이란發 외교 역풍… 벽두부터 이중 위기”    국제   
656  20200103                  중국, ‘원인불명 폐렴’ 속출… 사스 재발 공포 퍼져    국제   
657  20200103                 대만군 참모총장, 블랙호크 추락사고로 사망… 8명 숨져    국제   
658  20200103  [아직 살만한 세상] 새해 팁으로 ‘2020달러’ 놓고 간 손님… 여종업원의 눈물    국제   
659  20200103             질투에 눈이 멀어…‘청산가리 버블티’ 마신 베트남 간호사 사망    국제   

                                                  text  
0    문재인 대통령이 1일 오전 ‘2019년을 빛낸 의인’ 7명과 함께 서울 아차산에 올...  
1    문희상 국회의장이 2020년 새해 첫날인 1일 오전 서울 동작구 국립서울

In [ ]:
df.to_csv('kuk')